In [2]:
from langchain_ollama import OllamaLLM

# Initialize the Ollama model (e.g., llama3.2, gemma2, phi3, etc.)
llm = OllamaLLM(
    model="gemma3:270m",        # or "gemma2", "mistral", "phi3", etc.
    temperature=0.7,
    base_url="http://localhost:11436",  # default, change if needed
)

# List of prompts you want to run in batch
prompts = [
    "Explain quantum computing in simple terms.",
    "Write a haiku about artificial intelligence.",
    "Translate this to French: 'The quick brown fox jumps over the lazy dog.'",
    "Summarize the theory of relativity in one paragraph.",
    "Tell me a short joke about programmers.",
]

print("Running batch inference...\n")

# This invokes all prompts in parallel (batch mode)
responses = llm.batch(prompts, config={"max_concurrent": 10})  # optional concurrency limit

# Print results
for prompt, response in zip(prompts, responses):
    print(f"Prompt: {prompt}")
    print(f"Response: {response.strip()}\n")
    print("-" * 80)

Running batch inference...

Prompt: Explain quantum computing in simple terms.
Response: Imagine a light switch. It can be either on (representing a 1) or off (representing a 0). Quantum computing uses a light switch that can be both on and off at the same time, a process called superposition and entanglement.

Here's how it works:

* **Superposition:** Instead of being in a definite state (like a light switch), a quantum computer can be in multiple states simultaneously. Think of it like a coin spinning in the air – it's neither heads nor tails until it lands.
* **Entanglement:** This is like having two of these coins linked together. If one is heads, the other instantly becomes tails, even if they're miles apart. This creates a powerful connection that allows the computer to explore a vast number of possibilities simultaneously.

**Why is this powerful?**

Because of superposition and entanglement, quantum computers can:

* **Solve complex problems that are impossible for regular com

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
import json
from typing import Literal

# Choose one of these (make sure it's pulled with `ollama pull <name>`)
MODEL = "gemma3:270m"          # ← super fast + accurate for sentiment
# MODEL = "llama3.2:3b"      # also excellent
# MODEL = "gemma2:9b-it"     # highest accuracy

llm = OllamaLLM(
    model=MODEL,
    temperature=0.0,         # deterministic = consistent sentiment labels
    format="json",           # forces valid JSON output (works great with phi3/gemma2/llama3+)
)

# Zero-shot prompt that forces clean JSON output
sentiment_prompt = PromptTemplate.from_template(
    """Analyze the sentiment of the following customer review.

Return ONLY a valid JSON object with exactly these three keys:
{{"sentiment": "positive" | "negative" | "neutral", 
  "confidence": a number between 0.0 and 1.0,
  "explanation": short reason in 1 sentence}}

Text: "{text}"

JSON:"""
)

# Example dataset (replace with your own list, DataFrame column, etc.)
reviews = [
    "I absolutely love this product! Best purchase ever.",
    "It's okay, nothing special. Does the job.",
    "Terrible customer service and the item broke in half.",
    "Super fast shipping and great quality!",
    "Not bad, but the color is different from the photo.",
    "Worst experience ever. Never buying again.",
    "Meh. Expected more for the price.",
]

print(f"Running batch sentiment analysis on {len(reviews)} reviews using {MODEL}...\n")

# This runs all prompts in parallel (true batching)
chain = sentiment_prompt | llm
results = chain.batch([{"text": r} for r in reviews], config={"max_concurrency": 20})

# Parse and display clean results
print("Results:\n" + "="*60)
for i, raw_output in enumerate(results):
    try:
        # Clean common artifacts (some models add ```json or extra text)
        json_str = raw_output.strip()
        if json_str.startswith("```json"):
            json_str = json_str[7:-3]
        elif json_str.startswith("```"):
            json_str = json_str[3:-3]
        
        data = json.loads(json_str)
        
        sentiment = data.get("sentiment", "unknown").upper()
        confidence = data.get("confidence", 0.0)
        explanation = data.get("explanation", "")
        
        print(f"{i+1:2d}. [{sentiment:8}] (conf: {confidence:.2f}) → {reviews[i][:60]}...")
        if explanation:
            print(f"     ↳ {explanation}\n")
            
    except json.JSONDecodeError as e:
        print(f"{i+1:2d}. [PARSE ERROR] Raw output: {raw_output}\n")

print("Batch sentiment analysis completed!")

Running batch sentiment analysis on 7 reviews using gemma3:270m...

Results:
 1. [POSITIVE] (conf: 0.90) → I absolutely love this product! Best purchase ever....
     ↳ The customer is highly satisfied with the product and expresses strong positive feelings.

 2. [POSITIVE] (conf: 0.00) → It's okay, nothing special. Does the job....
 3. [NEGATIVE] (conf: 0.95) → Terrible customer service and the item broke in half....
     ↳ The customer service was extremely negative, with a broken item and a negative rating.

 4. [POSITIVE] (conf: 0.90) → Super fast shipping and great quality!...
     ↳ The customer is very satisfied with the fast shipping and quality of the product.

 5. [POSITIVE] (conf: 0.80) → Not bad, but the color is different from the photo....
 6. [NEGATIVE] (conf: 0.95) → Worst experience ever. Never buying again....
     ↳ The customer expresses strong negative sentiment, indicating a negative experience.

 7. [NEGATIVE] (conf: 0.90) → Meh. Expected more for the price....
 